# US Representative Voting Patterns and Funding Sources

*Exploratory analysis of how funding influences voting habits in Congress*

Part 1 of Exploratory Analysis: Pulling FEC funding data through API

In [21]:
import requests as rq # to pull data from FEC API
import pandas as pd # for data exploration
from keys import api_key_fec # holds secure API KEY
import sqlite3 as sq # to pull data from sqlite database
import difflib # fuzzy matching data from multiple tables

***

### Creating a reference table of registered committees - to use with further API requests

**Conditions for filters**
1. Active in 2012 (election cycle)
2. Filing Frequency: Monthly or Quarterly Filer
3. Designation: All but Unauthorized
4. Organization Type: All
5. Committee_Type: All but Party, Nonqualified/Qualified/National Non-Federal Account

**Steps**
1. Set up blank list and for loop for API pagination 
2. Get request from API with above filters added to url
3. Extend all pages pulled into one list and convert to DataFrame



In [2]:
# example request to test result
# committees_rq = rq.get(f'https://api.open.fec.gov/v1/committees/?page=1&per_page=100&year=2012&cycle=2012&filing_frequency=M&filing_frequency=Q&designation=A&designation=J&designation=P&designation=B&designation=D&organization_type=C&organization_type=L&organization_type=M&organization_type=T&organization_type=V&organization_type=W&committee_type=C&committee_type=D&committee_type=E&committee_type=H&committee_type=I&committee_type=N&committee_type=O&committee_type=P&committee_type=Q&committee_type=S&committee_type=U&sort=name&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false&api_key={api_key_fec}')

# print(committees_rq.status_code)
# committees = committees_rq.json()
# committees

In [3]:
# for loop to pull all ten pages of requests and combine into one DF

committees_all_pages = []
for i in range(1,11):
    committees_rq = rq.get(f'https://api.open.fec.gov/v1/committees/?page={i}&per_page=100&year=2012&cycle=2012&filing_frequency=M&filing_frequency=Q&designation=A&designation=J&designation=P&designation=B&designation=D&organization_type=C&organization_type=L&organization_type=M&organization_type=T&organization_type=V&organization_type=W&committee_type=C&committee_type=D&committee_type=E&committee_type=H&committee_type=I&committee_type=N&committee_type=O&committee_type=P&committee_type=Q&committee_type=S&committee_type=U&sort=name&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false&api_key={api_key_fec}')
    
    committees = committees_rq.json()
    committees_all_pages.append(committees)

In [4]:
#merge all results into one DF
blank_list = []
for i in range(0,10):
    blank_list.extend(committees_all_pages[i]['results'] )  

In [5]:
committees_df = pd.DataFrame(blank_list)

In [6]:
committees_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   affiliated_committee_name  976 non-null    object
 1   candidate_ids              991 non-null    object
 2   committee_id               991 non-null    object
 3   committee_type             991 non-null    object
 4   committee_type_full        991 non-null    object
 5   cycles                     991 non-null    object
 6   designation                991 non-null    object
 7   designation_full           991 non-null    object
 8   filing_frequency           991 non-null    object
 9   first_f1_date              991 non-null    object
 10  first_file_date            991 non-null    object
 11  last_f1_date               991 non-null    object
 12  last_file_date             991 non-null    object
 13  name                       991 non-null    object
 14  organizati

In [7]:
committees_df.head(5)

,affiliated_committee_name,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_f1_date,...,last_file_date,name,organization_type,organization_type_full,party,party_full,sponsor_candidate_ids,sponsor_candidate_list,state,treasurer_name
0,3M COMPANY,[],C00084475,Q,PAC - Qualified,"[1978, 1980, 1982, 1984, 1986, 1988, 1990, 199...",B,Lobbyist/Registrant PAC,M,1978-01-10,...,2023-08-14,3M COMPANY PAC,C,Corporation,PAC,None,None,[],MN,"TENNESSEN, TRACY"
1,ABBOTT LABORATORIES,[],C00040279,Q,PAC - Qualified,"[1976, 1978, 1980, 1982, 1984, 1986, 1988, 199...",B,Lobbyist/Registrant PAC,M,1976-04-19,...,2023-08-18,ABBOTT LABORATORIES EMPLOYEE POLITICAL ACTION ...,C,Corporation,UNK,UNKNOWN,None,[],IL,"GUERRERO, CYNTHIA"
2,ABBVIE,[],C00536573,Q,PAC - Qualified,"[2012, 2014, 2016, 2018, 2020, 2022, 2024]",B,Lobbyist/Registrant PAC,M,2012-11-19,...,2023-08-15,ABBVIE POLITICAL ACTION COMMITTEE,C,Corporation,None,None,None,[],IL,"KOLENDA, TIMOTHY"
3,"ABIOMED, INC.",[],C00426445,Q,PAC - Qualified,"[2006, 2008, 2010, 2012, 2014, 2016, 2018, 202...",B,Lobbyist/Registrant PAC,Q,2006-07-17,...,2023-08-16,ABIOMED INC POLITICAL ACTION COMMITTEE (ABIOME...,C,Corporation,None,None,None,[],MA,"LIMOLI, JUSTIN"
4,"ACCENTURE, INC.",[],C00300707,Q,PAC - Qualified,"[1996, 1998, 2000, 2002, 2004, 2006, 2008, 201...",B,Lobbyist/Registrant PAC,Q,1995-01-19,...,2023-07-20,ACCENTURE INC. POLITICAL ACTION COMMITTEE,C,Corporation,None,None,None,[],DC,"NICHOLSON, MATTHEW R."


#### Committee Referential Table Clean-Up

1. Dropping unnecessary columns (blank/mostly null or not relevant to analysis)
2. Renaming remaining columns
3. Checking for any nulls, exploring basic description of data before export
4. Saving as csv
5. Create list of committee ids for funding API requests

In [8]:
# dropping columns: 

committees_df.drop(columns=['candidate_ids','first_f1_date','first_file_date','last_f1_date','last_file_date','sponsor_candidate_ids','sponsor_candidate_list','filing_frequency','party','party_full','committee_type','organization_type','designation','cycles','treasurer_name'],inplace=True)

In [9]:
committees_df.columns

Index(['affiliated_committee_name', 'committee_id', 'committee_type_full',
       'designation_full', 'name', 'organization_type_full', 'state'],
      dtype='object')

In [10]:
# renaming column names 
committees_df.columns = ['affiliated_committee_name', 'committee_id', 'committee_type', 'designation_type', 'name', 'organization_type', 'state']

In [11]:
committees_df.head(1)

,affiliated_committee_name,committee_id,committee_type,designation_type,name,organization_type,state
0,3M COMPANY,C00084475,PAC - Qualified,Lobbyist/Registrant PAC,3M COMPANY PAC,Corporation,MN


In [12]:
committees_df.shape

(991, 7)

In [13]:
committees_df.describe(include='all')

,affiliated_committee_name,committee_id,committee_type,designation_type,name,organization_type,state
count,976,991,991,991,991,991,991
unique,943,991,2,4,991,6,45
top,NONE,C00084475,PAC - Qualified,Lobbyist/Registrant PAC,3M COMPANY PAC,Corporation,DC
freq,20,1,948,986,1,577,367


In [14]:
committees_df.dtypes

affiliated_committee_name    object
committee_id                 object
committee_type               object
designation_type             object
name                         object
organization_type            object
state                        object
dtype: object

In [15]:
committees_df.to_csv('./datasets/committees.csv',sep=',',index=False)

In [16]:
committee_id_list = committees_df['committee_id']
committee_id_list

0      C00084475
1      C00040279
2      C00536573
3      C00426445
4      C00300707
         ...    
986    C00329474
987    C00279455
988    C00025536
989    C00399386
990    C00235036
Name: committee_id, Length: 991, dtype: object

***

### Creating a reference table of registered candidates - to use with further API requests

**Filters for Candidate Search API**
1. Cycle - 2012
2. Election Year - 2012
3. Office - H, S
4. Year - 2012
5. Has Raised Funds - True

In [2]:
# API Request

members_rq = rq.get(f'https://api.open.fec.gov/v1/candidates/search/?page=1&per_page=100&cycle=2012&election_year=2012&office=H&office=S&year=2012&has_raised_funds=true&sort=name&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false&api_key={api_key_fec}')

print(members_rq.status_code)
members_json = members_rq.json()

members_json

200


{'api_version': '1.0',
 'pagination': {'count': 2120, 'page': 1, 'pages': 22, 'per_page': 100},
 'results': [{'active_through': 2018,
   'candidate_id': 'S2UT00229',
   'candidate_inactive': False,
   'candidate_status': 'P',
   'cycles': [2012, 2014, 2016, 2018, 2020],
   'district': '00',
   'district_number': 0,
   'election_districts': ['00', '00'],
   'election_years': [2012, 2018],
   'federal_funds_flag': False,
   'first_file_date': '2012-02-08',
   'has_raised_funds': True,
   'inactive_election_years': None,
   'incumbent_challenge': 'O',
   'incumbent_challenge_full': 'Open seat',
   'last_f2_date': '2018-04-23',
   'last_file_date': '2018-04-23',
   'load_date': '2019-03-27T16:02:41',
   'name': 'AALDERS, TIMOTHY NOEL',
   'office': 'S',
   'office_full': 'Senate',
   'party': 'CON',
   'party_full': 'CONSTITUTION PARTY',
   'principal_committees': [{'affiliated_committee_name': 'NONE',
     'candidate_ids': ['S2UT00229'],
     'committee_id': 'C00678300',
     'committee_t

In [3]:
# for loop to pull all twenty-two pages of requests and combine into one DF

candidates_all_pages = []
for i in range(1,23):
    candidates_rq = rq.get(f'https://api.open.fec.gov/v1/candidates/search/?page={i}&per_page=100&cycle=2012&election_year=2012&office=H&office=S&year=2012&has_raised_funds=true&sort=name&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false&api_key={api_key_fec}')
    
    candidates = candidates_rq.json()
    candidates_all_pages.append(candidates)

In [4]:
#merge all results into one DF
blank_list = []
for i in range(0,22):
    blank_list.extend(candidates_all_pages[i]['results'])  

In [5]:
candidates_df = pd.DataFrame(blank_list)

In [6]:
candidates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2120 entries, 0 to 2119
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   active_through            2120 non-null   int64  
 1   candidate_id              2120 non-null   object 
 2   candidate_inactive        2120 non-null   bool   
 3   candidate_status          2120 non-null   object 
 4   cycles                    2120 non-null   object 
 5   district                  2118 non-null   object 
 6   district_number           2118 non-null   float64
 7   election_districts        2120 non-null   object 
 8   election_years            2120 non-null   object 
 9   federal_funds_flag        2120 non-null   bool   
 10  first_file_date           2105 non-null   object 
 11  has_raised_funds          2120 non-null   bool   
 12  inactive_election_years   330 non-null    object 
 13  incumbent_challenge       2111 non-null   object 
 14  incumben

In [7]:
candidates_df.drop(columns=['active_through', 'cycles', 'candidate_inactive', 'district_number', 'election_districts', 'election_years', 'federal_funds_flag', 'first_file_date', 'inactive_election_years', 'incumbent_challenge', 'last_f2_date', 'last_file_date', 'load_date', 'party', 'office'],inplace=True)

In [8]:
candidates_df.shape

(2120, 10)

In [9]:
candidates_df.columns = ['candidate_id', 'status', 'district', 'has_raised_funds', 'incumbent_challenge', 'name', 'office', 'party', 'committees', 'state']

In [10]:
candidates_df.columns

Index(['candidate_id', 'status', 'district', 'has_raised_funds',
       'incumbent_challenge', 'name', 'office', 'party', 'committees',
       'state'],
      dtype='object')

In [11]:
candidates_df.head()

,candidate_id,status,district,has_raised_funds,incumbent_challenge,name,office,party,committees,state
0,S2UT00229,P,00,True,Open seat,"AALDERS, TIMOTHY NOEL",Senate,CONSTITUTION PARTY,"[{'affiliated_committee_name': 'NONE', 'candid...",UT
1,H2CA01110,P,01,True,Challenger,"AANESTAD, SAMUEL",House,REPUBLICAN PARTY,"[{'affiliated_committee_name': None, 'candidat...",CA
2,H2AZ02279,P,02,True,Challenger,"ABOUD, PAULA ANN",House,DEMOCRATIC PARTY,"[{'affiliated_committee_name': None, 'candidat...",AZ
3,H4NY07011,N,05,True,Challenger,"ACKERMAN, GARY L.",House,DEMOCRATIC PARTY,"[{'affiliated_committee_name': 'NONE', 'candid...",NY
4,H2CA25176,P,25,True,Challenger,"ACOSTA, DANTE",House,REPUBLICAN PARTY,"[{'affiliated_committee_name': None, 'candidat...",CA


In [12]:
candidates_df = candidates_df.loc[candidates_df['has_raised_funds'] == True]

In [13]:
candidates_df = candidates_df.apply(lambda row: row[candidates_df['party'].isin(['REPUBLICAN PARTY','DEMOCRATIC PARTY', 'INDEPENDENT'])])

In [14]:
candidates_df.shape

(1992, 10)

In [15]:
candidates_df.to_csv('./datasets/candidates.csv',sep=',',index=False)

#### Match Names from Members and Candidates

1. Link members of the 113th congress to their candidate information from the FEC
2. Add congress id to candidate table as a foreign key

In [67]:
conn = sq.connect('congress.db')
members = pd.read_sql_query("SELECT * FROM members", conn)

In [68]:
members.head(2)

,chamber,icpsr_id,district,state,party,name,congress_id,nominate_dim1,nominate_dim2
0,President,99911,0,USA,100,"OBAMA, Barack",O000167,-0.358,-0.197
1,House,20300,1,AL,200,"BONNER, Jr., Josiah Robins (Jo)",B001244,0.367,0.513


In [69]:
members['name'] = members['name'].str.lower()
members.head()

,chamber,icpsr_id,district,state,party,name,congress_id,nominate_dim1,nominate_dim2
0,President,99911,0,USA,100,"obama, barack",O000167,-0.358,-0.197
1,House,20300,1,AL,200,"bonner, jr., josiah robins (jo)",B001244,0.367,0.513
2,House,20301,3,AL,200,"rogers, mike dennis",R000575,0.363,0.455
3,House,21102,7,AL,100,"sewell, terri",S001185,-0.396,0.398
4,House,21192,2,AL,200,"roby, martha",R000591,0.362,0.658


In [70]:
candidates_df.head(2)

,candidate_id,status,district,has_raised_funds,incumbent_challenge,name,office,party,committees,state
1,H2CA01110,P,01,True,Challenger,"aanestad, samuel",House,REPUBLICAN PARTY,"[{'affiliated_committee_name': None, 'candidat...",CA
2,H2AZ02279,P,02,True,Challenger,"aboud, paula ann",House,DEMOCRATIC PARTY,"[{'affiliated_committee_name': None, 'candidat...",AZ


In [71]:
candidates_df['name'] = candidates_df['name'].str.lower()
candidates_df.head()

,candidate_id,status,district,has_raised_funds,incumbent_challenge,name,office,party,committees,state
1,H2CA01110,P,01,True,Challenger,"aanestad, samuel",House,REPUBLICAN PARTY,"[{'affiliated_committee_name': None, 'candidat...",CA
2,H2AZ02279,P,02,True,Challenger,"aboud, paula ann",House,DEMOCRATIC PARTY,"[{'affiliated_committee_name': None, 'candidat...",AZ
3,H4NY07011,N,05,True,Challenger,"ackerman, gary l.",House,DEMOCRATIC PARTY,"[{'affiliated_committee_name': 'NONE', 'candid...",NY
4,H2CA25176,P,25,True,Challenger,"acosta, dante",House,REPUBLICAN PARTY,"[{'affiliated_committee_name': None, 'candidat...",CA
6,H6FL11175,N,14,True,Challenger,"adams, eddie jr",House,REPUBLICAN PARTY,"[{'affiliated_committee_name': None, 'candidat...",FL


In [72]:
# creating a duplicate column of names in members table
members['name_match'] = members['name']

In [73]:
# convert member name in members table to the name it most closely matches in candidate_df
members['name_match'] = members['name_match'].apply(lambda x: difflib.get_close_matches(x, candidates_df['name'], n=1))

In [83]:
pd.set_option('display.max_rows', None)
members

,chamber,icpsr_id,district,state,party,name,congress_id,nominate_dim1,nominate_dim2,name_match
0,President,99911,0,USA,100,"obama, barack",O000167,-0.358,-0.197,[]
1,House,20300,1,AL,200,"bonner, jr., josiah robins (jo)",B001244,0.367,0.513,"[bonner, josiah robins jr.]"
2,House,20301,3,AL,200,"rogers, mike dennis",R000575,0.363,0.455,"[rogers, michael j]"
3,House,21102,7,AL,100,"sewell, terri",S001185,-0.396,0.398,"[sewell, terri a.]"
4,House,21192,2,AL,200,"roby, martha",R000591,0.362,0.658,"[roby, martha]"
5,House,21193,5,AL,200,"brooks, mo",B001274,0.652,-0.417,"[brooks, mo]"
6,House,21376,1,AL,200,"byrne, bradley",B001289,0.610,0.250,"[schneider, bradley s.]"
7,House,29301,6,AL,200,"bachus, spencer t., iii",B000013,0.387,0.228,"[bachus, spencer rep.]"
8,House,29701,4,AL,200,"aderholt, robert",A000055,0.386,0.561,"[aderholt, robert b. rep.]"
9,House,14066,1,AK,200,"young, donald edwin",Y000033,0.283,0.022,"[young, donald e]"


In [41]:
members.dtypes

chamber           object
icpsr_id           int64
district          object
state             object
party              int64
name              object
congress_id       object
nominate_dim1    float64
nominate_dim2    float64
name_match        object
dtype: object

In [84]:
# members_fec = candidates_df.merge(members, left_on)

### Set up API Funding Request Template

**Conditions for Schedule A Filters**
1. Contributor Type - Committee
2. Two Year Transaction Period - 2012 (end of election cycle)
3. Recipient Committee Type - House, Senate

**Conditions for Schedule B Filters**
1. Spender Committee Designation - all but unauthorized
2. Spender Committee Org Type - all
3. Spender Committee_Type - all but Party, nonqualified/qualified/national
4. Two Year Transaction Period - 2012 (end of election cycle)

#### Schedule A - Candidate Receipts

In [114]:
# testing schedule A pull request for member of the house --> John Boehner : H0OH08029 and contributions from the NRA

# Pulling first page to get disbursement date and index for pagination while loop

boehner_rq = rq.get(f'https://api.open.fec.gov/v1/schedules/schedule_a/?committee_id=C00237198&contributor_id=C00053553&contributor_type=committee&two_year_transaction_period=2012&recipient_committee_type=H&recipient_committee_type=S&per_page=100&sort=-contribution_receipt_date&sort_hide_null=false&sort_null_only=false&api_key={api_key_fec}')

print(boehner_rq.status_code)
boehner_json = boehner_rq.json()

boehner_page_test = boehner_json['pagination']
boehner_index_test = boehner_page_test['last_indexes']
boehner_index_test

200


{'last_contribution_receipt_date': '2011-09-20',
 'last_index': '4051520131189259936'}

In [115]:
boehner_json

{'api_version': '1.0',
 'pagination': {'count': 1,
  'last_indexes': {'last_contribution_receipt_date': '2011-09-20',
   'last_index': '4051520131189259936'},
  'pages': 1,
  'per_page': 100},
 'results': [{'amendment_indicator': 'N',
   'amendment_indicator_desc': 'NO CHANGE',
   'back_reference_schedule_name': None,
   'back_reference_transaction_id': None,
   'candidate_first_name': None,
   'candidate_id': None,
   'candidate_last_name': None,
   'candidate_middle_name': None,
   'candidate_name': None,
   'candidate_office': None,
   'candidate_office_district': None,
   'candidate_office_full': None,
   'candidate_office_state': None,
   'candidate_office_state_full': None,
   'candidate_prefix': None,
   'candidate_suffix': None,
   'committee': {'affiliated_committee_name': 'BOEHNER FOR SPEAKER FOJB-NRCC',
    'candidate_ids': ['H0OH08029'],
    'city': 'WEST CHESTER',
    'committee_id': 'C00237198',
    'committee_type': 'H',
    'committee_type_full': 'House',
    'cycle': 2

In [116]:
boehner_list = boehner_json['results']
boehner_list

[{'amendment_indicator': 'N',
  'amendment_indicator_desc': 'NO CHANGE',
  'back_reference_schedule_name': None,
  'back_reference_transaction_id': None,
  'candidate_first_name': None,
  'candidate_id': None,
  'candidate_last_name': None,
  'candidate_middle_name': None,
  'candidate_name': None,
  'candidate_office': None,
  'candidate_office_district': None,
  'candidate_office_full': None,
  'candidate_office_state': None,
  'candidate_office_state_full': None,
  'candidate_prefix': None,
  'candidate_suffix': None,
  'committee': {'affiliated_committee_name': 'BOEHNER FOR SPEAKER FOJB-NRCC',
   'candidate_ids': ['H0OH08029'],
   'city': 'WEST CHESTER',
   'committee_id': 'C00237198',
   'committee_type': 'H',
   'committee_type_full': 'House',
   'cycle': 2012,
   'cycles': [1990,
    1992,
    1994,
    1996,
    1998,
    2000,
    2002,
    2004,
    2006,
    2008,
    2010,
    2012,
    2014,
    2016,
    2018],
   'cycles_has_activity': [1990,
    1992,
    1994,
    1996

In [118]:
boehner_dict1 = boehner_list[0]

In [120]:
boehner_dict1

{'amendment_indicator': 'N',
 'amendment_indicator_desc': 'NO CHANGE',
 'back_reference_schedule_name': None,
 'back_reference_transaction_id': None,
 'candidate_first_name': None,
 'candidate_id': None,
 'candidate_last_name': None,
 'candidate_middle_name': None,
 'candidate_name': None,
 'candidate_office': None,
 'candidate_office_district': None,
 'candidate_office_full': None,
 'candidate_office_state': None,
 'candidate_office_state_full': None,
 'candidate_prefix': None,
 'candidate_suffix': None,
 'committee': {'affiliated_committee_name': 'BOEHNER FOR SPEAKER FOJB-NRCC',
  'candidate_ids': ['H0OH08029'],
  'city': 'WEST CHESTER',
  'committee_id': 'C00237198',
  'committee_type': 'H',
  'committee_type_full': 'House',
  'cycle': 2012,
  'cycles': [1990,
   1992,
   1994,
   1996,
   1998,
   2000,
   2002,
   2004,
   2006,
   2008,
   2010,
   2012,
   2014,
   2016,
   2018],
  'cycles_has_activity': [1990,
   1992,
   1994,
   1996,
   1998,
   2000,
   2002,
   2004,
   2

In [117]:
# create while loop for pagination capture

# variables set by first page request, count set to track number of pages requested
last_contribution_receipt_date = '2012-09-21'
last_index = '4102220121167751830'
count = 1

while True:
    try:
        # api request that inserts api key variable, as well as the last index and disbursement date from the first page request
        boehner_rq_loop = rq.get(f'https://api.open.fec.gov/v1/schedules/schedule_a/?contributor_id=C00053553&contributor_type=committee&two_year_transaction_period=2012&recipient_committee_type=H&recipient_committee_type=S&per_page=100&sort=-contribution_receipt_date&sort_hide_null=false&sort_null_only=false&api_key={api_key_fec}&last_disbursement_date={last_contribution_receipt_date}')

        # check status during while loop
        print(f'Requesting page {count}')
        boehner_rq_loop.raise_for_status()

        # convert request into json file
        boehner_json_loop = boehner_rq_loop.json()

        # move through json to pull new last index and disbursement dates
        boehner_page_loop = boehner_json_loop['pagination']
        boehner_index_loop = boehner_page_loop['last_indexes']
        boehner_index = boehner_index_loop['last_index']
        boehner_contribution_date = boehner_index_loop['last_contribution_receipt_date']
       
        # convert loop into a list and append to boehner_list
        boehner_list_loop = list(boehner_json_loop)
        boehner_list.append(boehner_list_loop)
        
        # add to counter
        count += 1
        
    except TypeError:
        print('No more valid pages to loop through')
        break



Requesting page 1
Requesting page 2
Requesting page 3
Requesting page 4
Requesting page 5
Requesting page 6
Requesting page 7
Requesting page 8
Requesting page 9
Requesting page 10
Requesting page 11
Requesting page 12
Requesting page 13
Requesting page 14
Requesting page 15
Requesting page 16
Requesting page 17
Requesting page 18
Requesting page 19
Requesting page 20
Requesting page 21
Requesting page 22
Requesting page 23
Requesting page 24
Requesting page 25
Requesting page 26
Requesting page 27
Requesting page 28
Requesting page 29
Requesting page 30
Requesting page 31
Requesting page 32
Requesting page 33
Requesting page 34
Requesting page 35
Requesting page 36
Requesting page 37
Requesting page 38
Requesting page 39
Requesting page 40
Requesting page 41
Requesting page 42
Requesting page 43
Requesting page 44
Requesting page 45


KeyboardInterrupt: 

#### Schedule B - Committee Fund Designation

In [85]:
# testing schedule b pull request for one committee id --> NRA : C00053553

nra_rq = rq.get(f'https://api.open.fec.gov/v1/schedules/schedule_b/?committee_id=C00053553&spender_committee_designation=A&spender_committee_designation=J&spender_committee_designation=P&spender_committee_designation=B&spender_committee_designation=D&spender_committee_org_type=C&spender_committee_org_type=L&spender_committee_org_type=M&spender_committee_org_type=T&spender_committee_org_type=V&spender_committee_org_type=W&spender_committee_type=C&spender_committee_type=D&spender_committee_type=E&spender_committee_type=H&spender_committee_type=I&spender_committee_type=N&spender_committee_type=O&spender_committee_type=P&spender_committee_type=Q&spender_committee_type=S&spender_committee_type=U&spender_committee_type=V&spender_committee_type=W&two_year_transaction_period=2012&per_page=100&sort=-disbursement_date&sort_hide_null=false&sort_null_only=false&api_key={api_key_fec}')

print(nra_rq.status_code)
nra_json = nra_rq.json()

200


In [86]:
nra_page_test = nra_json['pagination']
nra_index_test = nra_page_test['last_indexes']
nra_index_test['last_index']
nra_index_test['last_disbursement_date']

'2012-10-31'

In [87]:
nra_list = nra_json['results']
nra_list

[{'amendment_indicator': 'A',
  'amendment_indicator_desc': 'ADD',
  'back_reference_schedule_id': None,
  'back_reference_transaction_id': None,
  'beneficiary_committee_name': None,
  'candidate_first_name': None,
  'candidate_id': None,
  'candidate_last_name': None,
  'candidate_middle_name': None,
  'candidate_name': None,
  'candidate_office': None,
  'candidate_office_description': None,
  'candidate_office_district': None,
  'candidate_office_state': None,
  'candidate_office_state_full': None,
  'candidate_prefix': None,
  'candidate_suffix': None,
  'category_code': '001',
  'category_code_full': 'Administrative/Salary/Overhead Expenses ',
  'comm_dt': None,
  'committee': {'affiliated_committee_name': 'THE NATIONAL RIFLE ASSOCIATION OF AMERICA',
   'candidate_ids': [],
   'city': 'FAIRFAX',
   'committee_id': 'C00053553',
   'committee_type': 'Q',
   'committee_type_full': 'PAC - Qualified',
   'cycle': 2012,
   'cycles': [1976,
    1978,
    1980,
    1982,
    1984,
    19

In [88]:
# create while loop for pagination capture

# variables set by first page request, count set to track number of pages requested
last_disbursement_date = '2012-10-31'
last_index = '4040220131185979472'
count = 1

while True:
    try:
        # api request that inserts api key variable, as well as the last index and disbursement date from the first page request
        nra_rq_loop = rq.get(f'https://api.open.fec.gov/v1/schedules/schedule_b/?committee_id=C00053553&spender_committee_designation=A&spender_committee_designation=J&spender_committee_designation=P&spender_committee_designation=B&spender_committee_designation=D&spender_committee_org_type=C&spender_committee_org_type=L&spender_committee_org_type=M&spender_committee_org_type=T&spender_committee_org_type=V&spender_committee_org_type=W&spender_committee_type=C&spender_committee_type=D&spender_committee_type=E&spender_committee_type=H&spender_committee_type=I&spender_committee_type=N&spender_committee_type=O&spender_committee_type=P&spender_committee_type=Q&spender_committee_type=S&spender_committee_type=U&spender_committee_type=V&spender_committee_type=W&two_year_transaction_period=2012&per_page=100&sort=-disbursement_date&sort_hide_null=false&sort_null_only=false&api_key={api_key_fec}&last_index={last_index}&last_disbursement_date={last_disbursement_date}')

        # check status during while loop
        print(f'Requesting page {count}')
        nra_rq_loop.raise_for_status()

        # convert request into json file
        nra_json_loop = nra_rq_loop.json()

        # move through json to pull new last index and disbursement dates
        nra_page_loop = nra_json_loop['pagination']
        nra_index_loop = nra_page_loop['last_indexes']
        last_index = nra_index_loop['last_index']
        last_disbursement_date = nra_index_loop['last_disbursement_date']
       
        # convert loop into a list and append to nra_list
        nra_list_loop = list(nra_json_loop)
        nra_list.append(nra_list_loop)
        
        # add to counter
        count += 1
        
    except TypeError:
        print('No more valid pages to loop through')
        break

Requesting page 1
Requesting page 2
Requesting page 3
Requesting page 4
Requesting page 5
Requesting page 6
Requesting page 7
Requesting page 8
Requesting page 9
Requesting page 10
Requesting page 11
Requesting page 12
Requesting page 13
Requesting page 14
Requesting page 15
Requesting page 16
Requesting page 17
Requesting page 18
Requesting page 19
Requesting page 20
Requesting page 21
Requesting page 22
Requesting page 23
Requesting page 24
Requesting page 25
Requesting page 26
Requesting page 27
Requesting page 28
Requesting page 29
No more valid pages to loop through


In [89]:
nra_list

[{'amendment_indicator': 'A',
  'amendment_indicator_desc': 'ADD',
  'back_reference_schedule_id': None,
  'back_reference_transaction_id': None,
  'beneficiary_committee_name': None,
  'candidate_first_name': None,
  'candidate_id': None,
  'candidate_last_name': None,
  'candidate_middle_name': None,
  'candidate_name': None,
  'candidate_office': None,
  'candidate_office_description': None,
  'candidate_office_district': None,
  'candidate_office_state': None,
  'candidate_office_state_full': None,
  'candidate_prefix': None,
  'candidate_suffix': None,
  'category_code': '001',
  'category_code_full': 'Administrative/Salary/Overhead Expenses ',
  'comm_dt': None,
  'committee': {'affiliated_committee_name': 'THE NATIONAL RIFLE ASSOCIATION OF AMERICA',
   'candidate_ids': [],
   'city': 'FAIRFAX',
   'committee_id': 'C00053553',
   'committee_type': 'Q',
   'committee_type_full': 'PAC - Qualified',
   'cycle': 2012,
   'cycles': [1976,
    1978,
    1980,
    1982,
    1984,
    19

In [90]:
del nra_list[100:]

In [91]:
[type(d) for d in nra_list]

[dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict]

In [92]:
# delete empty list items
nra_no_committee = [{k: v for k, v in d.items() if k != 'committee'} for d in nra_list]

In [93]:
nra_df = pd.DataFrame(nra_no_committee)

In [94]:
nra_df

,amendment_indicator,amendment_indicator_desc,back_reference_schedule_id,back_reference_transaction_id,beneficiary_committee_name,candidate_first_name,candidate_id,candidate_last_name,candidate_middle_name,candidate_name,...,schedule_type,schedule_type_full,semi_annual_bundled_refund,spender_committee_designation,spender_committee_org_type,spender_committee_type,sub_id,transaction_id,two_year_transaction_period,unused_recipient_committee_id
0,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774329,49465437,2012,None
1,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774328,49465436,2012,None
2,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774327,49465435,2012,None
3,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774325,49465433,2012,None
4,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774321,49465429,2012,None
5,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774317,49256205,2012,None
6,A,ADD,None,None,None,THOMAS,None,GREASON,A,"GREASON, THOMAS A",...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774316,49255981,2012,None
7,A,ADD,None,None,None,DAVID,None,RAMADAN,None,"RAMADAN, DAVID",...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774315,49255175,2012,None
8,A,ADD,None,None,None,RICHARD,None,ANDERSON,None,"ANDERSON, RICHARD VA DEL.",...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774314,49255173,2012,None
9,A,ADD,None,None,None,None,None,None,None,None,...,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774313,49255172,2012,None


In [97]:
nra_df.shape

(100, 79)

In [95]:
nra_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 79 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   amendment_indicator                    100 non-null    object 
 1   amendment_indicator_desc               100 non-null    object 
 2   back_reference_schedule_id             0 non-null      object 
 3   back_reference_transaction_id          0 non-null      object 
 4   beneficiary_committee_name             14 non-null     object 
 5   candidate_first_name                   57 non-null     object 
 6   candidate_id                           13 non-null     object 
 7   candidate_last_name                    57 non-null     object 
 8   candidate_middle_name                  2 non-null      object 
 9   candidate_name                         57 non-null     object 
 10  candidate_office                       56 non-null     object 
 11  candida

In [99]:
#dropping null/unnecessary columns (1st round)
nra_df.drop(columns=['back_reference_schedule_id', 'back_reference_transaction_id', 'candidate_first_name', 'candidate_last_name', 'candidate_middle_name', 'candidate_suffix', 'comm_dt', 'conduit_committee_city', 'conduit_committee_name', 'conduit_committee_state', 'conduit_committee_street1', 'conduit_committee_street2', 'conduit_committee_zip', 'memo_code', 'memo_code_full', 'national_committee_nonfederal_account', 'original_sub_id', 'payee_employer', 'payee_first_name', 'payee_last_name', 'payee_middle_name', 'payee_occupation', 'payee_prefix', 'payee_suffix', 'ref_disp_excess_flg'], inplace=True)

In [102]:
pd.set_option('display.max_columns', None)
nra_df.head(100)

,amendment_indicator,amendment_indicator_desc,beneficiary_committee_name,candidate_id,candidate_name,candidate_office,candidate_office_description,candidate_office_district,candidate_office_state,candidate_office_state_full,candidate_prefix,category_code,category_code_full,committee_id,disbursement_amount,disbursement_date,disbursement_description,disbursement_purpose_category,disbursement_type,disbursement_type_description,election_type,election_type_full,entity_type,entity_type_desc,fec_election_type_desc,fec_election_year,file_number,filing_form,image_number,line_number,line_number_label,link_id,load_date,memo_text,memoed_subtotal,pdf_url,recipient_city,recipient_committee,recipient_committee_id,recipient_name,recipient_state,recipient_zip,report_type,report_year,schedule_type,schedule_type_full,semi_annual_bundled_refund,spender_committee_designation,spender_committee_org_type,spender_committee_type,sub_id,transaction_id,two_year_transaction_period,unused_recipient_committee_id
0,A,ADD,None,None,None,None,None,None,None,None,None,001,Administrative/Salary/Overhead Expenses,C00053553,49.64,2012-12-31,CREDIT CARD FEES,OTHER,None,None,None,None,ORG,ORGANIZATION,None,None,852382,F3X,13960611624,21B,Other Federal Operating Expenditures,4013120131180363071,2017-05-31T20:12:38.059660,CREDIT CARD FEES,False,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,SALEM,None,None,"PAYMENTECH, INC.",NH,03079,YE,2012,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774329,49465437,2012,None
1,A,ADD,None,None,None,None,None,None,None,None,None,001,Administrative/Salary/Overhead Expenses,C00053553,4.03,2012-12-31,CREDIT CARD FEES,OTHER,None,None,None,None,ORG,ORGANIZATION,None,None,852382,F3X,13960611623,21B,Other Federal Operating Expenditures,4013120131180363071,2017-05-31T20:12:38.059660,CREDIT CARD FEES,False,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,PHOENIX,None,None,AMERICAN EXPRESS COMPANY,AZ,85072,YE,2012,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774328,49465436,2012,None
2,A,ADD,None,None,None,None,None,None,None,None,None,001,Administrative/Salary/Overhead Expenses,C00053553,2.00,2012-12-31,BANK FEE,OTHER,None,None,None,None,ORG,ORGANIZATION,None,None,852382,F3X,13960611623,21B,Other Federal Operating Expenditures,4013120131180363071,2017-05-31T20:12:38.059660,BANK FEE,False,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,FAIRFAX,None,None,BB&T BANK,VA,22030,YE,2012,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774327,49465435,2012,None
3,A,ADD,None,None,None,None,None,None,None,None,None,001,Administrative/Salary/Overhead Expenses,C00053553,3.00,2012-12-31,BANK FEE,OTHER,None,None,None,None,ORG,ORGANIZATION,None,None,852382,F3X,13960611622,21B,Other Federal Operating Expenditures,4013120131180363071,2017-05-31T20:12:38.059660,BANK FEE,False,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,ST. LOUIS,None,None,BANK OF AMERICA - MO,MO,63106,YE,2012,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774325,49465433,2012,None
4,A,ADD,None,None,None,None,None,None,None,None,None,001,Administrative/Salary/Overhead Expenses,C00053553,12.00,2012-12-31,BANK FEE,OTHER,None,None,None,None,ORG,ORGANIZATION,None,None,852382,F3X,13960611621,21B,Other Federal Operating Expenditures,4013120131180363071,2017-05-31T20:12:38.059660,BANK FEE,False,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,ALLIANCE,None,None,BANK OF THE WEST,NE,69301,YE,2012,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774321,49465429,2012,None
5,A,ADD,None,None,None,None,None,None,None,None,None,011,Political Contributions,C00053553,1000.00,2012-12-31,None,OTHER,None,None,None,None,COM,OTHER COMMITTEE,None,None,852382,F3X,13960611630,29,Other Disbursements,4013120131180363071,2017-05-31T20:12:38.059660,None,False,http://docquery.fec.gov/cgi-bin/fecimg/?139606...,COLONIAL HEIGHTS,None,None,MAJORITY LEADER PAC,VA,23834,YE,2012,SB,ITEMIZED DISBURSEMENTS,0.00,B,M,Q,4020520131180774317,49256205,2012,None
6,A,ADD,None,None,"GREASON, THOMAS A",H,HOUSE,32,VA

In [103]:
#dropping additional unnecessary columns (2nd round)
nra_df.drop(columns=['amendment_indicator_desc', 'candidate_office_state_full', 'candidate_prefix', 'category_code', 'entity_type_desc', 'fec_election_type_desc', 'fec_election_year', 'file_number', 'filing_form', 'image_number', 'line_number', 'link_id', 'load_date', 'memoed_subtotal', 'pdf_url', 'report_type', 'report_year', 'schedule_type', 'schedule_type_full', 'semi_annual_bundled_refund', 'spender_committee_designation', 'sub_id', 'two_year_transaction_period'], inplace=True)

In [107]:
nra_df.head(100)

,amendment_indicator,beneficiary_committee_name,candidate_id,candidate_name,candidate_office,candidate_office_description,candidate_office_district,candidate_office_state,category_code_full,committee_id,disbursement_amount,disbursement_date,disbursement_description,disbursement_purpose_category,disbursement_type,disbursement_type_description,election_type,election_type_full,entity_type,line_number_label,memo_text,recipient_city,recipient_committee,recipient_committee_id,recipient_name,recipient_state,recipient_zip,spender_committee_org_type,spender_committee_type,transaction_id,unused_recipient_committee_id
0,A,None,None,None,None,None,None,None,Administrative/Salary/Overhead Expenses,C00053553,49.64,2012-12-31,CREDIT CARD FEES,OTHER,None,None,None,None,ORG,Other Federal Operating Expenditures,CREDIT CARD FEES,SALEM,None,None,"PAYMENTECH, INC.",NH,03079,M,Q,49465437,None
1,A,None,None,None,None,None,None,None,Administrative/Salary/Overhead Expenses,C00053553,4.03,2012-12-31,CREDIT CARD FEES,OTHER,None,None,None,None,ORG,Other Federal Operating Expenditures,CREDIT CARD FEES,PHOENIX,None,None,AMERICAN EXPRESS COMPANY,AZ,85072,M,Q,49465436,None
2,A,None,None,None,None,None,None,None,Administrative/Salary/Overhead Expenses,C00053553,2.00,2012-12-31,BANK FEE,OTHER,None,None,None,None,ORG,Other Federal Operating Expenditures,BANK FEE,FAIRFAX,None,None,BB&T BANK,VA,22030,M,Q,49465435,None
3,A,None,None,None,None,None,None,None,Administrative/Salary/Overhead Expenses,C00053553,3.00,2012-12-31,BANK FEE,OTHER,None,None,None,None,ORG,Other Federal Operating Expenditures,BANK FEE,ST. LOUIS,None,None,BANK OF AMERICA - MO,MO,63106,M,Q,49465433,None
4,A,None,None,None,None,None,None,None,Administrative/Salary/Overhead Expenses,C00053553,12.00,2012-12-31,BANK FEE,OTHER,None,None,None,None,ORG,Other Federal Operating Expenditures,BANK FEE,ALLIANCE,None,None,BANK OF THE WEST,NE,69301,M,Q,49465429,None
5,A,None,None,None,None,None,None,None,Political Contributions,C00053553,1000.00,2012-12-31,None,OTHER,None,None,None,None,COM,Other Disbursements,None,COLONIAL HEIGHTS,None,None,MAJORITY LEADER PAC,VA,23834,M,Q,49256205,None
6,A,None,None,"GREASON, THOMAS A",H,HOUSE,32,VA,Political Contributions,C00053553,1000.00,2012-12-31,"THOMAS GREASON, STATE HOUSE 32ND VA",OTHER,None,None,G2013,None,COM,Other Disbursements,"THOMAS GREASON, STATE HOUSE 32ND VA",POTOMAC FALLS,None,None,FRIENDS OF TAG GREASON,VA,20165,M,Q,49255981,None
7,A,None,None,"RAMADAN, DAVID",H,HOUSE,87,VA,Political Contributions,C00053553,500.00,2012-12-31,"DAVID RAMADAN, STATE HOUSE 87TH VA",OTHER,None,None,G2013,None,COM,Other Disbursements,"DAVID RAMADAN, STATE HOUSE 87TH VA",CHANTILLY,None,None,FRIENDS OF DAVID RAMADAN,VA,20152,M,Q,49255175,None
8,A,None,None,"ANDERSON, RICHARD VA DEL.",H,HOUSE,51,VA,Political Contributions,C00053553,250.00,2012-12-31,"RICHARD ANDERSON, STATE HOUSE 51ST VA",OTHER,None,None,G2013,None,COM,Other Disbursements,"RICHARD ANDERSON, STATE HOUSE 51ST VA",WOODBRIDGE,None,None,FRIENDS OF RICH ANDERSON,VA,22195,M,Q,49255173,None
9,A,None,None,None,None,None,None,None,Political Contributions,C00053553,500.00,2012-12-31,None,OTHER,None,None,None,None,PTY,Other Disbursements,None,WILLIAMSBURG,None,None,VIRGINIA SENATE REPUBLICAN CAUCUS,VA,23187,M,Q,49255172,None


In [108]:
#dropping additional unnecessary columns (3rd round)
nra_df.drop(columns=['candidate_office_description', 'disbursement_type', 'disbursement_type_description', 'recipient_committee', 'election_type', 'election_type_full', 'recipient_state', 'recipient_zip'], inplace=True)

In [110]:
nra_df.columns

Index(['amendment_indicator', 'beneficiary_committee_name', 'candidate_id',
       'candidate_name', 'candidate_office', 'candidate_office_district',
       'candidate_office_state', 'category_code_full', 'committee_id',
       'disbursement_amount', 'disbursement_date', 'disbursement_description',
       'disbursement_purpose_category', 'entity_type', 'line_number_label',
       'memo_text', 'recipient_city', 'recipient_committee_id',
       'recipient_name', 'spender_committee_org_type',
       'spender_committee_type', 'transaction_id',
       'unused_recipient_committee_id'],
      dtype='object')

In [111]:
#dropping additional unnecessary columns (4th round)
nra_df.drop(columns=['beneficiary_committee_name', 'candidate_office_district', 'candidate_office_state', 'line_number_label', 'memo_text', 'recipient_city'], inplace=True)

In [112]:
nra_df.shape

(100, 17)

In [113]:
nra_df.columns

Index(['amendment_indicator', 'candidate_id', 'candidate_name',
       'candidate_office', 'category_code_full', 'committee_id',
       'disbursement_amount', 'disbursement_date', 'disbursement_description',
       'disbursement_purpose_category', 'entity_type',
       'recipient_committee_id', 'recipient_name',
       'spender_committee_org_type', 'spender_committee_type',
       'transaction_id', 'unused_recipient_committee_id'],
      dtype='object')

In [96]:
nra_df['candidate_name'].value_counts()

BALTIMORE, CHIP IA REP.      2
KLIMESH, MICHAEL             2
MOORE, BRIAN IA REP.         2
KENYON, JAMES                2
KAUFMANN, JEFF IA REP.       2
STANERSON, QUENTIN           2
GREASON, THOMAS A            1
HALL, DAN MN SEN.            1
PETERSON, ROZ                1
LOHMER, KATHY MN REP.        1
SANDERS, TIM MN REP.         1
BENZ, MANDY                  1
MURRAY, RICH MN REP.         1
BLY, BREANNA                 1
BANAIAN, KING MN REP.        1
KRUSE, BENJAMIN MN SEN.      1
DOWNEY, KEITH                1
HOUSLEY, KARIN               1
WOLF, PAM MN SEN.            1
LOUDENBECK, AMY WI REP.      1
SPARKS, DAN MN SEN.          1
NELSON, CARLA MN SEN.        1
PEDERSON, JOHN MN SEN.       1
RUUD, CARRIE                 1
HARSDORF, SHEILA E.          1
VOS, ROBIN WI REP.           1
RAMADAN, DAVID               1
AUGUST, TYLER WI REP.        1
ANDERSON, RICHARD VA DEL.    1
MACEWEN, DREW WA REP.        1
VELA, FILEMON                1
GIBSON, CHRISTOPHER MR.      1
LUCAS, F